In [14]:
#import the functions
from wildcatpy.api_calls import WildcatApi
from wildcatpy.src.helper_functions import check_bounds
import json

In [16]:
test = dataExtractor("test")
cleaner_file = test.get_extractor_json("track_extractor")

In [17]:
cleaner_file

[{'full_key': ['extracted', 'content', 0, 'headers'],
  'all_columns': ['entityId', 'entityType', 'projectId', 'projectName'],
  'explode_values': []},
 {'full_key': ['extracted', 'content', 1, 'GeoFeature'],
  'all_columns': ['featureType', 'length', 'startWhen', 'endWhen'],
  'explode_values': []},
 {'full_key': ['extracted', 'content', 1, 'GeoFeature', 'agent'],
  'all_columns': ['agentName'],
  'explode_values': []}]

In [3]:
def find_values_to_extract(extractor, 
                           values_to_extract = None,
                           full_key = None,
                          ):
    if values_to_extract == None:
        values_to_extract = []
    if full_key == None:
        full_key = []
    for key,value in extractor.items():
        if key.isnumeric(): #check bc there can be integers
            key = int(key)
        if key == "extract_values":
            values_to_extract.append({"full_key":full_key, "all_columns": value, "explode_values": []})
        elif key == "explode_values":
            values_to_extract.append({"full_key":full_key, "all_columns": [], "explode_values": value})

        else:
            new_key = full_key + [key]
            values_to_extract.extend(find_values_to_extract(value,
                                     full_key=new_key)
                                    )
    return values_to_extract

def recurGet(d, ks): 
    head, *tail = ks 
    return recurGet(d[head], tail) if tail else d[head]

class dataExtractor:
    def __init__(self,
                 input_data
                ):
        self.input_data = input_data
        self.data = input_data
        
    def deeper_in_nested(self,keys):
        self.data = recurGet(self.data,keys)
    def select_columns(self,keep_cols):
        self.data = [
            {key:item} for row in self.data\
                       for key,item in row.items()\
                       if key in keep_cols
                    ]       
    def list_to_pd(self):
        return pd.DataFrame(self.data)
    def flatten_data(self):
        self.data = sum(self.data, [])
    def get_list_dict(self):
        return self.data
    def get_list_values(self):
        return [[_ for _ in row.values()] for row in self.data]
    def get_extractor_json(self, file_name):
        with open(f"wildcatpy/extractors/{file_name}.json", "r") as f:
            return find_values_to_extract(json.load(f))
    def iterate_extractor(self,extractor_file_name):
        new_data = []
        for row in self.data:
            new_data.append(self.extract_with_extractor(row,extractor_file_name))
        self.data = new_data
    def extract_with_extractor(self,
                               data_for_extract,
                               extractor_file_name
                              ): 
        new_data = {}
        additional = [{}]#if it isn't made in the extraction of exploding columns
        for extract_val in self.get_extractor_json(extractor_file_name):
            if len(extract_val["all_columns"]) > 0:
                full_key = extract_val["full_key"]
                all_columns = extract_val["all_columns"]
                data = recurGet(data_for_extract, full_key)
                new_data = {**new_data, **{key:data[key] for key in all_columns}}
        for extract_val in self.get_extractor_json(extractor_file_name):
            if len(extract_val["explode_values"]) > 0:
                full_key = extract_val["full_key"]
                print(full_key)
                data = recurGet(data_for_extract, full_key)
                all_extr_columns = extract_val["explode_values"]
                if len(data) > 0:
                    additional = [{col:row[col] for col in all_extr_columns} for row in data]
        return [{**new_data, **add} for add in additional]
        






### 1.0 Login  (done)

Needed before this step
* 
  


In [1]:
username= "jobvancreij"
password = ""

In [2]:
#store this api_call object
#This is needed for future calls
api_call = WildcatApi(username,password)

NameError: name 'WildcatApi' is not defined

### 1.1: Get groups (done)

Needed before this step
* 1.1 Login


In [6]:
info = api_call.get_groups()

{'filters': {'dateTimeRange': {'from': '1900-01-01T00:00:00-00:00', 'to': '9999-12-31T23:59:59-00:00'}, 'entities': ['Observation', 'track']}, 'options': {'start': 1, 'pageLength': 0}}
req to https://focus.sensingclues.org/api/search/all/facets success


In [7]:
help(api_call.get_groups)

Help on method get_groups in module wildcatpy.api_calls:

get_groups() method of wildcatpy.api_calls.WildcatApi instance



In [8]:
import pandas as pd
cleaner = dataExtractor(info)
print(info)
cleaner.iterate_extractor("groups_extractor")

cleaner.select_columns(["name"])
df = cleaner.list_to_pd()
display(df)

TypeError: 'dataExtractor' object is not iterable

In [9]:
info = cleaner.get_list_values()
groups = info[0]
print(groups)

TypeError: 'dataExtractor' object is not iterable

### 1.2: get-observations
Needed before this step
* 1.1 Login
* 1.2 Get groups

### 1.3: get-track-metadata
Needed before this step
* 1.1 Login
* 1.2 Get groups

In [10]:
def make_query(bounds=None,
               operator = None,
               timerange=None,
               type_analysis=None,
               groups=None,
               page_info=None,
               end_time = "T23:59:59-00:00", #if someone has to make very specific calls change this
               start_time = "T00:00:00-00:00"
              ):
    output = { 
        (["filters"]["geoQuery"]["mapBounds"])
    
    }
    query_object = {"filters": {"geoQuery": {} }}
    if bounds != None:
        query_object["filters"]["geoQuery"]["mapBounds"] = bounds
    if operator != None:
        query_object["filters"]["geoQuery"]["operator"] = operator
    if timerange != None:
        query_object["filters"]["dateTimeRange"] = { 
                     "to": f"{timerange[1]}{end_time}", #"2009-10-10T12:00:00-05:00",
                      "from": f"{timerange[1]}{start_time}"
                      }
    if type_analysis != None:
        query_object["filters"]["entities"] = type_analysis

    if groups != None:
        query_object["filters"]["dataSources"] = groups
    if page_info != None:
        query_object["options"] = {
            "start": page_info[0],
             "page_length": page_info[1]
        }
        
    return query_object

        



<>:11: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:11: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/var/folders/ym/vg2hn__n4h55zb6g8m66zg7w0000gn/T/ipykernel_10819/1256664948.py:11: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  (["filters"]["geoQuery"]["mapBounds"])


In [11]:
test = ["a"]
test.pop(0)

'a'

In [12]:
def make_nested_dict(output_prep,
                     output_dict = None,
                    ):
    if output_dict == None:
        output_dict = {} 
    for prep in output_prep:
        keys = prep[1]
        values = prep[0]
    
        

In [13]:
def make_nested_dict(value, 
                     keys,
                     output_dict = None,
                    ):
    if output_dict == None:
        output_dict = {}
    key = keys.pop(0)
    if (len(keys) > 0 and output_dict.get(key, False)): #if more then one key and dict key exists
        output_dict[key] = make_nested_dict(value, keys,output_dict[key])
    elif (len(keys) > 0 and not  output_dict.get(key, False)):#if more then one key and dict key doesn't exist   
        output_dict[key] = make_nested_dict(value, keys,{})     
    else: #if only 1 key left finalize the dict
        output_dict[key] = value
    return output_dict

   
    

In [ ]:
def make_query(bounds=None,
               operator = None,
               date_from=None,
               date_to=None,
               type_analysis=None,
               groups=None,
               page_nbr=None,
               page_length=None,
               end_time = "T23:59:59-00:00", #if someone has to make very specific calls change this
               start_time = "T00:00:00-00:00"
              ):
    page_nbr = None if (page_nbr == None or page_length == None) else page_nbr* page_length +1
    time_from = None if date_from == None else f"{date_from}{start_time}" #add date and time 
    time_to = None if date_to == None else f"{date_to}{end_time}" #add date and time 

    # shows the value and where it should be placed in the dictionary
    # For new vars just add (<varname, [<cols><in><final><dict>])
    output_prep = [
        (bounds, ["filters","geoQuery","mapBounds"]),
        (operator, ["filters", "geoQuery", "operator"]),
        (time_from, ["filters", "dateTimeRange", "from"]),
        (time_to, ["filters", "dateTimeRange", "to"]),
        (type_analysis, ["filters", "entities"]),
        (groups, ["filters", "dataSources"]),
        (page_nbr , ["options", "start"]),
        (page_length, ["options", "page_length"]),   
    ]
    
    final_output = [_ for _ in output_prep if _[0] != None] #only return non None vars
    #convert output_prep to query 
    query = make_nested_dict(*final_output[0]) #init query with first row (no output_dict yet)
    for row in final_output[1:]: 
        query = make_nested_dict(*row,query)#loop for the rest, send output dict so it is updated 
    return query
               

        





In [ ]:
output = make_query( time_from = "2020-01-1",
           time_to = "2022-12-31",
           type_analysis = ["Observation", "track"],
           page_nbr=1,
           page_length=0
          )
print(output)

In [ ]:
import math

def make_query(bounds,to_date,from_date,type_analysis,groups,page_nbr,_page_length):
    return  {
            "filters":
              {
                  "geoQuery":
                    {
                        "operator":"intersects",
                        "mapBounds":bounds,
                          "drawings":[]
                          },
                  "dateTimeRange":
                  {
                      "to": f"{to_date}T12:24:00-00:00", #"2009-10-10T12:00:00-05:00",
                      "from": f"{from_date}T00:00:00-00:00"
                  },
                  "entities":[type_analysis],
                  "dataSources": groups,
              },
          "options": {
              "start": page_nbr*_page_length+1,
              "pageLength": _page_length
          }
        }
    

class Test(WildcatApi):
    def _init(self):
        pass
    def track_extractor( self,
        groups,
        bounds = {"north":90,"east": 180,"west": -179,"south": -89},
        from_date = "1900-01-01",
        to_date = "9999-12-31",
        updateProgress = None
    ):
        return(self.general_api_test(groups,
                            "track",
                            "track_extractor",  
                            bounds,
                            from_date,
                            to_date,
                            updateProgress
                           ))
    def observation_extractor( self,
        groups,
        bounds = {"north":90,"east": 180,"west": -179,"south": -89},
        from_date = "1900-01-01",
        to_date = "9999-12-31",
        updateProgress = None
    ):
        df = self.general_api_test(groups,
                            "observation",
                            "observation_extractor",  
                            bounds,
                            from_date,
                            to_date,
                            updateProgress
                           )
        # fix filter for the concepts
        # transform concept id
        return df
    
    def general_api_test(
        self,
        groups,
        type_analysis,
        extractor_name,
        bounds = {"north":90,"east": 180,"west": -179,"south": -89},
        from_date = "1900-01-01",
        to_date = "9999-12-31",
        updateProgress = None,
        _page_length = 10


    ):
        output_data = []       
        extra_request = True
        first_iter = True
        page_nbr = 0     
        #fix timestamp!! 
        while extra_request:
            query = make_query(bounds,to_date,from_date,type_analysis,groups,page_nbr,_page_length)
            r = self.api_call("post","search/all/results", query)
            if first_iter:
                nbr_pages = math.ceil(r.json()["total"]/_page_length)
                if nbr_pages == 0:
                    break
            cleaner = dataExtractor(r.json())
            cleaner.deeper_in_nested(["results"])
            cleaner.iterate_extractor(extractor_name)
            cleaner.flatten_data()
            output_data.extend(cleaner.get_list_dict())
            if page_nbr == nbr_pages:
                break
            page_nbr +=1
        return pd.DataFrame(output_data)
            
      
            
    

In [ ]:
x = Test(username,password)

In [ ]:
data = x.track_extractor(groups,
        bounds = {"north":90,"east": 180,"west": -179,"south": -89},
        from_date = "2022-01-01",
                        )

In [ ]:
data

In [ ]:
data = x.general_api_test(groups,"Observation","observation_extractor")


In [ ]:
data

In [ ]:
def test(a=1,b=2,c=10,types=None):
    print(a)
    print(b)
    print(c)
    print(types)
    
def testb(**args):
    test(**args,types = "hao")
    
testb(a=5,b=10)